In [1]:
DATA_PREPARATION_VOTE_METHOD = "max_vote_window" # "max_vote_window" or "sum_and_normalize". Decides how to aggregate the predictions of the overlapping windows
PRETRAINED_MODEL_NAME_OR_PATH = "inception_v3"

In [2]:
import sys
import os

from tqdm import tqdm 
import wandb
import torch

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.amp import GradScaler, autocast
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR


if bool(os.environ.get("KAGGLE_URL_BASE", "")):
  import sys
  # running on kaggle
  sys.path.insert(0, "/kaggle/input/hsm-source-files")
else:
  # running locally
  sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))

from src.datasets.multi_spectrogram import MultiSpectrogramDataset
from src.utils.k_folds_creator import KFoldCreator
from src.utils.utils import get_models_save_path, set_seeds, get_raw_data_dir, get_processed_data_dir
from src.models.base_cnn import BaseCNN
from src.utils.constants import Constants 
from src.datasets.eeg_processor import EEGDataProcessor
from src.transforms.mixup import MixUp

/home/david/miniconda3/envs/aicomp/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/david/miniconda3/envs/aicomp/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened bec

In [3]:
wandb.login()

wandb: Currently logged in as: davidhodel (hms-hslu-aicomp-hs25) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
class CFG:
    seed = 42
    n_splits = 5
    data_path = get_raw_data_dir()
    train_eeg_spec_path = get_processed_data_dir() / "eeg_spectrograms" / "train" / "cwt"
    
    model_name = PRETRAINED_MODEL_NAME_OR_PATH
    target_size = 6 
    image_alignment = "stacked" # "stacked" or "paired"
    
    batch_size = 32
    num_workers = 8
    
    # Stage-specific settings
    stage1_epochs = 5
    stage1_lr = 1e-3
    
    stage2_epochs = 3
    stage2_lr = 5e-4
    
    patience = 5
    min_delta = 0.001
    
    dropout_p = 0.1
    
    img_size = (128, 256)
    # augmentations = ["gaussian_noise", "time_reversal", "time_masking", "frequency_masking"]
    augmentations = []
    mixup_alpha = 0.5
    mixup_prob = 1.0

set_seeds(42)

In [5]:
TARGETS = Constants.TARGETS

processor = EEGDataProcessor(raw_data_path=CFG.data_path, processed_data_path=get_processed_data_dir())

Processor initialized.
Raw data path: '/home/david/git/aicomp/data'
Processed data path: '/home/david/git/aicomp/data/processed'


In [6]:
def get_dataloaders(train_df, valid_df):
    """
    Updated to handle empty DataFrames safely.
    Returns None for the loader if the input DataFrame is empty.
    """
    train_loader = None
    valid_loader = None
    
    if train_df is not None and not train_df.empty:
        train_dataset = MultiSpectrogramDataset(
            train_df, TARGETS, CFG.data_path, CFG.img_size, CFG.train_eeg_spec_path, mode='train', apply_augmentations=CFG.augmentations
        )

        # Create custom collate function for training if MixUp is enabled
        train_collate_fn = MixUp(alpha=CFG.mixup_alpha, prob=CFG.mixup_prob) if CFG.mixup_prob > 0.0 else None

        train_loader = DataLoader(
            train_dataset,
            batch_size=CFG.batch_size,
            shuffle=True,
            num_workers=CFG.num_workers,
            pin_memory=False,
            drop_last=True,
            persistent_workers=True,
            collate_fn=train_collate_fn
        )
    
    if valid_df is not None and not valid_df.empty:
        valid_dataset = MultiSpectrogramDataset(
            valid_df, TARGETS, CFG.data_path, CFG.img_size, CFG.train_eeg_spec_path, mode='train', apply_augmentations=[]
        )
        
        valid_loader = DataLoader(
            valid_dataset,
            batch_size=CFG.batch_size,
            shuffle=False,
            num_workers=CFG.num_workers,
            pin_memory=False,
            drop_last=False,
            persistent_workers=True
        )
    
    return train_loader, valid_loader

In [7]:
def train_one_stage(fold, stage_name, train_df, valid_df, group_name, data_preparation_vote_method, starting_checkpoint=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    scaler = GradScaler(enabled=(device.type == "cuda"))
    
    if stage_name == "stage1":
        lr = CFG.stage1_lr
        epochs = CFG.stage1_epochs
    else:
        lr = CFG.stage2_lr
        epochs = CFG.stage2_epochs
    
    print(f"\n--- Starting {stage_name} | Fold {fold} ---")
    
    experiment_name = f"{group_name}_{stage_name}_fold{fold}"
    
    config = {
        # Model
        "architecture": CFG.model_name,
        "pretrained": True,
        "image_alignment": CFG.image_alignment,
        # Data
        "fold": fold,
        "stage": stage_name,
        "features": "multi_spectrograms",
        "window_selection": data_preparation_vote_method,
        "augmentations": CFG.augmentations,
        "mixup_alpha": CFG.mixup_alpha,
        "mixup_prob": CFG.mixup_prob,
        # Training
        "optimizer": "AdamW",
        "learning_rate": lr,
        "batch_size": CFG.batch_size,
        "epochs": epochs,
        "seed": CFG.seed,
        "scheduler": "CosineAnnealingLR",
        "dropout_p": CFG.dropout_p,
    }

    wandb_run = wandb.init(
        project="hms-aicomp-cnn-multispec",
        name=experiment_name,
        group=group_name,
        job_type=stage_name,
        tags=['two-stage', stage_name, f'fold{fold}'],
        config=config,
        reinit=True
    )

    model = BaseCNN(CFG.model_name, pretrained=True, num_classes=CFG.target_size, dropout_p=CFG.dropout_p, image_alignment=CFG.image_alignment)
    model.to(device)
    
    if starting_checkpoint:
        print(f"Loading weights from {starting_checkpoint}...")
        model.load_state_dict(torch.load(starting_checkpoint))

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    loss_fn = nn.KLDivLoss(reduction='batchmean')
    
    train_loader, valid_loader = get_dataloaders(train_df, valid_df)

    best_val_loss = float('inf')
    best_model_path = get_models_save_path() / "multi_spec_cnn_two_stage" / CFG.model_name / data_preparation_vote_method / f'{stage_name}_fold{fold}.pth'
    best_model_path.parent.mkdir(parents=True, exist_ok=True)
    
    patience_counter = 0

    for epoch in range(epochs):
        print(f"  --- Epoch {epoch+1}/{epochs} ---")

        model.train()
        train_loss = 0
        for images, labels in tqdm(train_loader, desc=f"{stage_name} Training E{epoch+1}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            with autocast(device_type=device.type, dtype=torch.float16):
                outputs = model(images)
                log_probs = F.log_softmax(outputs, dim=1)
                loss = loss_fn(log_probs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item() * images.size(0)

            wandb.log({"train/loss": loss.item()})

        train_loss /= len(train_loader.dataset)

        model.eval()
        valid_loss = 0
        with torch.no_grad():
            for images, labels in tqdm(valid_loader, desc=f"{stage_name} Validation"):
                images, labels = images.to(device), labels.to(device)

                with autocast(device_type=device.type, dtype=torch.float16):
                    outputs = model(images)
                    log_probs = F.log_softmax(outputs, dim=1)
                    loss = loss_fn(log_probs, labels)

                valid_loss += loss.item() * images.size(0)

        valid_loss /= len(valid_loader.dataset)
        
        epoch_lr = optimizer.param_groups[0]['lr']
        print(f"   Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Valid Loss = {valid_loss:.4f}, LR = {epoch_lr:.6f}")

        wandb.log({
            "epoch": epoch + 1,
            "train/epoch_loss": train_loss,
            "val/loss": valid_loss,
            "val/kl_div": valid_loss,
            "train/epoch_lr": epoch_lr
        })

        if valid_loss < best_val_loss - CFG.min_delta:
            best_val_loss = valid_loss
            patience_counter = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  New best model saved with validation loss: {best_val_loss:.4f}")
        else:
            patience_counter += 1
        
        if patience_counter >= CFG.patience:
            print(f"  Early stopping at epoch {epoch+1}")
            
        scheduler.step()

    return best_model_path, best_val_loss, wandb_run

In [8]:
def run_two_stage_training(df, data_preparation_vote_method):
    print("Total Votes Distribution (Head):")
    print(df['total_votes'].head())
    
    mask_low_votes = df['total_votes'] < 10
    mask_high_votes = df['total_votes'] >= 10
    
    print(f"Stage 1 Data (Low Votes < 10): {mask_low_votes.sum()} samples")
    print(f"Stage 2 Data (High Votes >= 10): {mask_high_votes.sum()} samples")
    
    group_name = f"twostage_{CFG.model_name}_multispec_dropout_{CFG.dropout_p}_mixup_alpha_{CFG.mixup_alpha}"
    
    all_oof_preds = []
    all_oof_labels = []
    fold_scores = []
    
    for fold in range(CFG.n_splits):
        print(f"\n{'='*20} Processing FOLD {fold} {'='*20}")
        
        valid_idx = df['fold'] == fold
        valid_df = df[valid_idx].reset_index(drop=True)
        
        valid_stage2_df = valid_df[valid_df['total_votes'] >= 10].reset_index(drop=True)

        # Stage 1: Train on low-vote data
        train_stage1 = df[(df['fold'] != fold) & mask_low_votes].reset_index(drop=True)
        
        stage1_path, _, _ = train_one_stage(
            fold=fold,
            stage_name="stage1",
            train_df=train_stage1,
            valid_df=valid_df,
            group_name=group_name,
            data_preparation_vote_method=data_preparation_vote_method
        )

        wandb.finish()

        # Stage 2: Train on high-vote data, starting from Stage 1 checkpoint
        train_stage2 = df[(df['fold'] != fold) & mask_high_votes].reset_index(drop=True)
        
        stage2_path, best_val_loss, wandb_stage2_run = train_one_stage(
            fold=fold,
            stage_name="stage2",
            train_df=train_stage2,
            valid_df=valid_stage2_df,
            starting_checkpoint=stage1_path,
            group_name=group_name,
            data_preparation_vote_method=data_preparation_vote_method
        )
        
        fold_scores.append(best_val_loss)

        # Generate OOF predictions using Stage 2 model
        print(f"   --- Generating OOF predictions for fold {fold} ---")
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        model = BaseCNN(CFG.model_name, pretrained=True, num_classes=CFG.target_size, dropout_p=CFG.dropout_p, image_alignment=CFG.image_alignment)
        model.load_state_dict(torch.load(stage2_path))
        model.to(device)
        model.eval()

        fold_oof_preds = []
        fold_oof_labels = []
        
        # Use full validation set for OOF predictions
        _, valid_loader = get_dataloaders(pd.DataFrame(), valid_df)

        with torch.no_grad():
            for images, labels in tqdm(valid_loader, desc=f"OOF Prediction Fold {fold}"):
                images = images.to(device)
                outputs = model(images)
                probs = F.softmax(outputs, dim=1).cpu()

                fold_oof_preds.append(probs)
                fold_oof_labels.append(labels.cpu())

        all_oof_preds.append(torch.cat(fold_oof_preds).numpy())
        all_oof_labels.append(torch.cat(fold_oof_labels).numpy())
        print(f"   Finished OOF predictions for fold {fold}")
        
        del model
        torch.cuda.empty_cache()

        # Upload artifact
        artifact = wandb.Artifact(wandb_stage2_run.name, type='model')
        artifact.add_file(stage2_path)
        wandb.log_artifact(artifact)
        wandb.finish()

    if all_oof_preds and all_oof_labels:
        print("\nCalculating final OOF score...")
        final_oof_preds = np.concatenate(all_oof_preds)
        final_oof_labels = np.concatenate(all_oof_labels)

        oof_preds_tensor = torch.tensor(final_oof_preds, dtype=torch.float32)
        oof_labels_tensor = torch.tensor(final_oof_labels, dtype=torch.float32)

        log_oof_preds_tensor = torch.log(oof_preds_tensor)

        kl_loss_fn = nn.KLDivLoss(reduction='batchmean')
        overall_oof_score = kl_loss_fn(log_oof_preds_tensor, oof_labels_tensor).item()

        print(f"\nOverall OOF KL Score: {overall_oof_score:.4f}")
        print(f"Average Fold Score: {np.mean(fold_scores):.4f}")
    else:
        print("\nCould not calculate OOF score because no predictions were generated.")
        overall_oof_score = None
        
    return overall_oof_score

## Run Training

In [9]:
print("Preparing data and creating folds...")
train_df = processor.process_data(vote_method=DATA_PREPARATION_VOTE_METHOD, skip_parquet=True)
print('Train shape:', train_df.shape)
print('Targets', list(TARGETS))

fold_creator = KFoldCreator(n_splits=CFG.n_splits, seed=CFG.seed)
train_df = fold_creator.create_folds(train_df, stratify_col='expert_consensus', group_col='patient_id')

print("Folds created. Value counts per fold:")
print(train_df['fold'].value_counts())

Preparing data and creating folds...
Starting EEG Data Processing Pipeline
Skipping Parquet file creation as requested.
Using 'max_vote_window' vote aggregation strategy.

Processed train data saved to '/home/david/git/aicomp/data/processed/train_processed.csv'.
Shape of the final dataframe: (17089, 13)

Pipeline finished successfully!
Train shape: (17089, 13)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
Folds created. Value counts per fold:
fold
0    4067
1    3658
2    3381
4    3358
3    2625
Name: count, dtype: int64


In [10]:
overall_oof_score = run_two_stage_training(train_df, DATA_PREPARATION_VOTE_METHOD)

Total Votes Distribution (Head):
0    12
1    14
2     1
3     1
4     2
Name: total_votes, dtype: int64
Stage 1 Data (Low Votes < 10): 11150 samples
Stage 2 Data (High Votes >= 10): 5939 samples

==================== Processing FOLD 0 ====================
Using device: cuda

--- Starting stage1 | Fold 0 ---


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


2025-12-03 16:57:33,518 :: timm.models._builder :: INFO :: Loading pretrained weights from Hugging Face hub (timm/inception_v3.tv_in1k)
2025-12-03 16:57:33,721 :: timm.models._hub :: INFO :: [timm/inception_v3.tv_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2025-12-03 16:57:33,757 :: timm.models._builder :: INFO :: Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.


  --- Epoch 1/5 ---


stage1 Validation:   0%|          | 0/128 [00:02<?, ?it/s]


...

   Epoch 1: Train Loss = 0.0125, Valid Loss = 0.0332, LR = 0.000500
  New best model saved with validation loss: 0.0332


/home/david/miniconda3/envs/aicomp/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


   --- Generating OOF predictions for fold 1 ---


2025-12-03 17:00:29,694 :: timm.models._builder :: INFO :: Loading pretrained weights from Hugging Face hub (timm/inception_v3.tv_in1k)
2025-12-03 17:00:29,819 :: timm.models._hub :: INFO :: [timm/inception_v3.tv_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2025-12-03 17:00:29,871 :: timm.models._builder :: INFO :: Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
OOF Prediction Fold 1:  37%|███▋      | 43/115 [00:45<01:16,  1.06s/it]


KeyboardInterrupt: 

In [ ]:
print(f"Overall OOF KL Score from training: {overall_oof_score:.4f}")